In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools, algorithms
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)

# Define multi-objective fitness to maximize accuracy, minimize complexity, and training time
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0))  # Maximize accuracy, minimize complexity & time
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Data Loading and Preprocessing
def load_data():
    print("Loading and preprocessing CIFAR-10 dataset...")
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Split training data into training and validation sets
    val_split = 0.1
    num_sample = 1000
    x_train = x_train[:num_sample]
    y_train = y_train[:num_sample]
    val_size = int(len(x_train) * val_split)
    x_val, y_val = x_train[:val_size], y_train[:val_size]
    x_train, y_train = x_train[val_size:], y_train[val_size:]

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data()
print("Data preparation complete.")
print("Training data shape:", x_train.shape)
print("Validation data shape:", x_val.shape)
print("Test data shape:", x_test.shape)

# Convert data to PyTorch format
train_dataset = TensorDataset(
    torch.tensor(x_train).permute(0, 3, 1, 2).float(), torch.tensor(np.argmax(y_train, axis=1)).long()
)
val_dataset = TensorDataset(
    torch.tensor(x_val).permute(0, 3, 1, 2).float(), torch.tensor(np.argmax(y_val, axis=1)).long()
)
test_dataset = TensorDataset(
    torch.tensor(x_test).permute(0, 3, 1, 2).float(), torch.tensor(np.argmax(y_test, axis=1)).long()
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, validation_loss):
        if validation_loss < self.best_loss - self.delta:
            self.best_loss = validation_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Neural Network Architecture Generation
def generate_individual():
    layers = []
    num_layers = random.randint(2, 4)  # Random number of layers
    for _ in range(num_layers):
        layer_type = random.choice(['conv', 'dense'])
        if layer_type == 'conv':
            layers.append({
                'type': 'conv',
                'filters': random.choice([16, 32, 64]),
                'activation': random.choice(['relu', 'tanh'])
            })
        elif layer_type == 'dense':
            layers.append({
                'type': 'dense',
                'units': random.choice([64, 128, 256]),
                'activation': random.choice(['relu', 'tanh'])
            })
    return layers

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

class CustomCNN(nn.Module):
    def __init__(self, architecture):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 3  # Input channels for CIFAR-10 (RGB)
        current_height, current_width = 32, 32  # CIFAR-10 image dimensions
        added_flatten = False
        input_size = None  # Initialize input_size to avoid referencing before assignment

        for layer in architecture:
            if layer['type'] == 'conv' and not added_flatten:
                self.layers.append(
                    nn.Conv2d(in_channels, layer['filters'], kernel_size=3, padding=1)
                )
                self.layers.append(nn.ReLU() if layer['activation'] == 'relu' else nn.Tanh())
                self.layers.append(nn.MaxPool2d(kernel_size=2))
                in_channels = layer['filters']
                # Update spatial dimensions after pooling
                current_height //= 2
                current_width //= 2
            elif layer['type'] == 'dense':
                if not added_flatten:
                    # Dynamically compute the input size for the first dense layer
                    input_size = current_height * current_width * in_channels
                    self.layers.append(nn.Flatten())
                    added_flatten = True
                # Add dense layers
                self.layers.append(nn.Linear(input_size, layer['units']))
                self.layers.append(nn.ReLU() if layer['activation'] == 'relu' else nn.Tanh())
                input_size = layer['units']

        # Ensure Flatten and Input Size Calculation
        if not added_flatten:
            self.layers.append(nn.Flatten())
            input_size = current_height * current_width * in_channels  # Compute for first Linear layer

        # Add output layer
        self.layers.append(nn.Linear(input_size, 10))  # 10 output classes for CIFAR-10

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def evaluate_cnn(individual):
    print(f"Evaluating individual: {individual}")  # Log the individual's architecture
    model = CustomCNN(individual)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    early_stopping = EarlyStopping(patience=5)

    for epoch in range(50):  # Train for 50 epochs or until early stopping
        model.train()
        train_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

        # Validation loss
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        early_stopping(val_loss)
        if early_stopping.early_stop:
            break

    # Calculate final accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            outputs = model(x_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    complexity = sum(p.numel() for p in model.parameters())
    print(f"Results for individual: Accuracy = {accuracy:.4f}, Complexity = {complexity}, Validation Loss = {val_loss:.4f}")  # Log results
    return accuracy, complexity, val_loss

toolbox.register("evaluate", evaluate_cnn)

# Evolutionary Algorithm
def evolutionary_algorithm(n_gen=6, pop_size=10, cxpb=0.7, mutpb=0.5):
    population = toolbox.population(n=pop_size)
    for gen in range(n_gen):
        print(f"\n-- Generation {gen} --")

        # Evaluate fitness of population
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            print(f"Individual Fitness: {fit}")  # Log fitness of each individual

        # Select offspring
        offspring = tools.selTournament(population, len(population), tournsize=3)
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for i in range(1, len(offspring), 2):  # Iterate over pairs of individuals
            if random.random() < cxpb:
                tools.cxTwoPoint(offspring[i - 1], offspring[i])

        # Apply mutation
        for mutant in offspring:
            if random.random() < mutpb:
                tools.mutShuffleIndexes(mutant, indpb=0.2)

        # Replace population with new offspring
        population[:] = offspring

    # Log final best individual
    best_individual = tools.selBest(population, 1)[0]
    print(f"Best Individual: {best_individual}, Fitness: {best_individual.fitness.values}")
    return best_individual

# Run Evolutionary Algorithm
best_architecture = evolutionary_algorithm()
print("Best architecture:", best_architecture)


Loading and preprocessing CIFAR-10 dataset...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Data preparation complete.
Training data shape: (900, 32, 32, 3)
Validation data shape: (100, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)

-- Generation 0 --
Evaluating individual: [{'type': 'conv', 'filters': 16, 'activation': 'tanh'}, {'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'conv', 'filters': 64, 'activation': 'relu'}, {'type': 'dense', 'units': 64, 'activation': 'relu'}]
Results for individual: Accuracy = 0.3500, Complexity = 78298, Validation Loss = 1.6581
Evaluating individual: [{'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'conv', 'filters': 64, 'activation': 'tanh'}]
Results for individual: Accuracy = 0.4500, Complexity = 50698, Validation Loss = 1.8692
Evaluating individual: [{'type': 'dense', 'units': 256, 'activation': 'tanh'}, {'type': 'conv', 'filters': 16, 'activation': 'tanh'}]
Results for individual: Accuracy = 0.2100, Complexity

###Larger Data SubSet###

Changing the Multifitness function for maximizing accuracy, minimizing complexity, and training time.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools, algorithms
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)

# Define multi-objective fitness to maximize accuracy, minimize complexity, and training time
creator.create("FitnessMulti", base.Fitness, weights=(2.0, -0.5, -1.0))  # Maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMulti)


# Data Loading, Preprocessing, and Splitting Function
def load_data(limit_samples=10000):
    """
    Load and preprocess CIFAR-10 dataset with a limited number of samples.
    Performs a 60/20/20 split on the limited dataset.

    Args:
        limit_samples (int): The total number of samples to use for training and validation.

    Returns:
        tuple: PyTorch DataLoaders for training, validation, and test datasets.
    """
    print("Loading and preprocessing CIFAR-10 dataset...")
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize pixel values to [0, 1]
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Limit the training dataset size
    x_train = x_train[:limit_samples]
    y_train = y_train[:limit_samples]

    # Perform 60/20/20 split
    total_samples = len(x_train)
    train_size = int(0.6 * total_samples)
    val_size = int(0.2 * total_samples)

    # Split the data
    x_train_split = x_train[:train_size]
    y_train_split = y_train[:train_size]
    x_val_split = x_train[train_size:train_size + val_size]
    y_val_split = y_train[train_size:train_size + val_size]
    x_test_split = x_test[:val_size]  # Optionally limit the test set for consistency
    y_test_split = y_test[:val_size]

    print(f"Data split into: {train_size} train, {val_size} validation, {len(x_test_split)} test samples.")

    # Convert to PyTorch tensors
    train_dataset = TensorDataset(
        torch.tensor(x_train_split).permute(0, 3, 1, 2).float(),
        torch.tensor(np.argmax(y_train_split, axis=1)).long()
    )
    val_dataset = TensorDataset(
        torch.tensor(x_val_split).permute(0, 3, 1, 2).float(),
        torch.tensor(np.argmax(y_val_split, axis=1)).long()
    )
    test_dataset = TensorDataset(
        torch.tensor(x_test_split).permute(0, 3, 1, 2).float(),
        torch.tensor(np.argmax(y_test_split, axis=1)).long()
    )

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

# Load data with a limit of 10,000 samples for training and validation
train_loader, val_loader, test_loader = load_data(limit_samples=10000)

# Print DataLoader sizes
print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

# Define Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, validation_loss):
        if validation_loss < self.best_loss - self.delta:
            self.best_loss = validation_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Neural Network Architecture Generation
def generate_individual():
    layers = []
    num_layers = random.randint(2, 4)  # Random number of layers
    for _ in range(num_layers):
        layer_type = random.choice(['conv', 'dense'])
        if layer_type == 'conv':
            layers.append({
                'type': 'conv',
                'filters': random.choice([16, 32, 64]),
                'activation': random.choice(['relu', 'tanh'])
            })
        elif layer_type == 'dense':
            layers.append({
                'type': 'dense',
                'units': random.choice([64, 128, 256]),
                'activation': random.choice(['relu', 'tanh'])
            })
    return layers

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

class CustomCNN(nn.Module):
    def __init__(self, architecture):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 3  # Input channels for CIFAR-10 (RGB)
        current_height, current_width = 32, 32  # CIFAR-10 image dimensions
        added_flatten = False
        input_size = None  # Initialize input_size to avoid referencing before assignment

        for layer in architecture:
            if layer['type'] == 'conv' and not added_flatten:
                self.layers.append(
                    nn.Conv2d(in_channels, layer['filters'], kernel_size=3, padding=1)
                )
                self.layers.append(nn.ReLU() if layer['activation'] == 'relu' else nn.Tanh())
                self.layers.append(nn.MaxPool2d(kernel_size=2))
                in_channels = layer['filters']
                # Update spatial dimensions after pooling
                current_height //= 2
                current_width //= 2
            elif layer['type'] == 'dense':
                if not added_flatten:
                    # Dynamically compute the input size for the first dense layer
                    input_size = current_height * current_width * in_channels
                    self.layers.append(nn.Flatten())
                    added_flatten = True
                # Add dense layers
                self.layers.append(nn.Linear(input_size, layer['units']))
                self.layers.append(nn.ReLU() if layer['activation'] == 'relu' else nn.Tanh())
                input_size = layer['units']

        # Ensure Flatten and Input Size Calculation
        if not added_flatten:
            self.layers.append(nn.Flatten())
            input_size = current_height * current_width * in_channels  # Compute for first Linear layer

        # Add output layer
        self.layers.append(nn.Linear(input_size, 10))  # 10 output classes for CIFAR-10

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def evaluate_cnn(individual):
    print(f"Evaluating individual: {individual}")  # Log the individual's architecture
    model = CustomCNN(individual)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    early_stopping = EarlyStopping(patience=10)

    for epoch in range(50):  # Train for 50 epochs or until early stopping
        model.train()
        train_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

        # Validation loss
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        early_stopping(val_loss)
        if early_stopping.early_stop:
            break

    # Calculate final accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            outputs = model(x_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    complexity = sum(p.numel() for p in model.parameters())
    print(f"Results for individual: Accuracy = {accuracy:.4f}, Complexity = {complexity}, Validation Loss = {val_loss:.4f}")  # Log results
    return accuracy, complexity, val_loss

toolbox.register("evaluate", evaluate_cnn)

# Evolutionary Algorithm
def evolutionary_algorithm(n_gen=6, pop_size=10, cxpb=0.7, mutpb=0.5):
    population = toolbox.population(n=pop_size)
    for gen in range(n_gen):
        print(f"\n-- Generation {gen} --")

        # Evaluate fitness of population
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            print(f"Individual Fitness: {fit}")  # Log fitness of each individual

        # Select offspring
        offspring = tools.selTournament(population, len(population), tournsize=3)
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for i in range(1, len(offspring), 2):  # Iterate over pairs of individuals
            if random.random() < cxpb:
                tools.cxTwoPoint(offspring[i - 1], offspring[i])

        # Apply mutation
        for mutant in offspring:
            if random.random() < mutpb:
                tools.mutShuffleIndexes(mutant, indpb=0.2)

        # Replace population with new offspring
        population[:] = offspring

    # Log final best individual
    best_individual = tools.selBest(population, 1)[0]
    print(f"Best Individual: {best_individual}, Fitness: {best_individual.fitness.values}")
    return best_individual

# Run Evolutionary Algorithm
best_architecture = evolutionary_algorithm()
print("Best architecture:", best_architecture)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Loading and preprocessing CIFAR-10 dataset...
Data preparation complete.
Training data shape: (4500, 32, 32, 3)
Validation data shape: (500, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)

-- Generation 0 --
Evaluating individual: [{'type': 'conv', 'filters': 16, 'activation': 'tanh'}, {'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'conv', 'filters': 64, 'activation': 'relu'}, {'type': 'dense', 'units': 64, 'activation': 'relu'}]
Results for individual: Accuracy = 0.5520, Complexity = 78298, Validation Loss = 1.6530
Evaluating individual: [{'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'conv', 'filters': 64, 'activation': 'tanh'}]
Results for individual: Accuracy = 0.5720, Complexity = 50698, Validation Loss = 1.4319
Evaluating individual: [{'type': 'dense', 'units': 256, 'activation': 'tanh'}, {'type': 'conv', 'filters': 16, 'activation': 'tanh'}]
Results for individual: Accuracy = 0.3340, Complexity = 789258, Validation Loss = 1.8469
Evaluating indiv

In [ ]:
def test_model(architecture):
    model = CustomCNN(architecture)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            outputs = model(x_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    print(f"Test Accuracy: {correct / total:.4f}")

test_model(best_architecture)


Test Accuracy: 0.0944
